# Assessment 2 MATH70094 Programming for Data Science Autumn 2025

# Kutadgu Gokalp Demirci - 06071571

# Question 2 (16 marks)

In this question you continue the analysis of the flight data using Python. 
 
Some helpful functions for this question are `merge`, `apply`, `groupby`, `dropna`.

## Code clarity (1 mark)

There is a famous saying among software developers that code is read more often than it is written. Marks will be awarded (or not awarded) based on the clarity of the code and appropriate use of comments.

## Part A

### Part A(i) (1 mark)

Load the contents of the csv files `flights.csv` and `planes.csv` as dataframes named `flights` and `planes`. Print the first 6 rows from each dataframe. How many flights are there in all?

In [97]:
import pandas as pd
import numpy as np

flights = pd.read_csv('flights.csv')
planes = pd.read_csv('planes.csv')

print(f"There are {flights.shape[0]} flights in the flights.csv dataset.")
print(f"There are {planes.shape[0]} planes in the planes.csv dataset.")

print(flights[0:5])
print(planes[0:5])

There are 336776 flights in the flights.csv dataset.
There are 3322 planes in the planes.csv dataset.
   year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0  2013      1    1     517.0             515        2.0     830.0   
1  2013      1    1     533.0             529        4.0     850.0   
2  2013      1    1     542.0             540        2.0     923.0   
3  2013      1    1     544.0             545       -1.0    1004.0   
4  2013      1    1     554.0             600       -6.0     812.0   

   sched_arr_time  arr_delay carrier  flight tailnum origin dest  air_time  \
0             819       11.0      UA    1545  N14228    EWR  IAH     227.0   
1             830       20.0      UA    1714  N24211    LGA  IAH     227.0   
2             850       33.0      AA    1141  N619AA    JFK  MIA     160.0   
3            1022      -18.0      B6     725  N804JB    JFK  BQN     183.0   
4             837      -25.0      DL     461  N668DN    LGA  ATL     116.0   

   dista

### Part A(ii) (1 mark)

Find the number of missing entries for each column in `flights`.

In [98]:
na_column_entries_flight = flights.isna().sum()
print(na_column_entries_flight)


# Double checking a missing entry.

#flights[flights['dep_time'].isna()].index[0:5]
#print(flights.loc[838])

year                 0
month                0
day                  0
dep_time          8255
sched_dep_time       0
dep_delay         8255
arr_time          8713
sched_arr_time       0
arr_delay         9430
carrier              0
flight               0
tailnum           2512
origin               0
dest                 0
air_time          9430
distance             0
hour                 0
minute               0
time_hour            0
dtype: int64


### Part A(iii) (2 marks)

Add a new column named `long_delayed_AA` to `flights`, which contains for a flight the value `True` if it is a flight by the carrier 'AA', is long haul (`distance` more than 1000 miles) and arrived with more than 30 minutes delay. Otherwise, it contains the value `False`. To check the results after adding the column, print the first 10 rows of `flights` including only the columns `carrier`, `distance`, `arr_delay` and `long_delayed_AA`. For how many flights is `long_delayed_AA` equal to `True`?  

In [99]:
#print(flights.shape)
flights['long_delayed_AA'] = (
    (flights['carrier'] == 'AA') &
    (flights['distance'] > 1000) &
    (flights['arr_delay'] > 30)
)

# Check if its created
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,long_delayed_AA
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,False
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,False
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,True
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,False
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,False


## Part B

There are many missing entries in the `arr_delay` column in `flights`. Your task is to impute as many missing values using information from other variables.

### Part B(i) (3 marks)

To identify which covariates affect `arr_delay`, create three dataframes from `flights` as follows:
- a dataframe named `monthly_stats` with two columns, which in each row contains for one of the 12 months the average arrival delay in that month as well as the total number of flights,
- a dataframe named `hourly_stats` with two columns, which in each row contains for one of the 24 departure hours in a day the average arrival delay in that departure hour as well as the total number of flights,
- a dataframe named `carrier_stats` with two columns, which in each row contains for one of the carriers the average arrival delay for that carrier as well as the total number of flights. 

Print all three newly created dataframes. Describe what you notice for each dataframe using one sentence.

In [111]:
flights_sub = flights.dropna(subset=['dep_time', 'arr_delay']).copy()
flights_sub['dep_hour'] = (flights_sub['dep_time'] // 100).astype(int)

hourly_stats = (flights_sub.groupby('dep_hour').agg(avg_arr_delay = ('arr_delay', 'mean'),
                                                total_flights = ('arr_delay', 'size')).round(2))
print(hourly_stats)


carrier_stats = (flights_sub.groupby('carrier').agg(avg_arr_delay = ('arr_delay', 'mean'),
                                                total_flights = ('arr_delay', 'size')).round(2))
print(carrier_stats)

          avg_arr_delay  total_flights
dep_hour                              
0                120.88            876
1                195.96            221
2                224.65             63
3                287.73             11
4                 -8.88            305
5                 -8.85           7230
6                 -7.09          22627
7                 -7.35          21409
8                 -4.03          26367
9                 -1.06          18820
10                -0.51          16031
11                 0.10          16712
12                 2.26          16024
13                 4.24          16845
14                 2.33          20394
15                 6.51          22703
16                 7.39          22236
17                11.33          22039
18                13.39          21453
19                14.59          19871
20                21.63          16097
21                35.60          10914
22                62.08           5461
23                90.74  

### Part B(iii) (2 marks)

Create a new dataframe `flights_imp` from `flights`. In the new dataframe, impute the missing `arr_delay` entries. To do so, fill in for a given row with a missing `arr_delay` entry the arrival delay averaged over all the flights with the same carrier within the same month and the same hour. After imputation, how many missing entries are left in the column `arr_delay`?

In [116]:
flights_imp = flights.copy()

# Extract departure hour
flights_imp['dep_hour'] = (flights_imp['dep_time'] // 100).astype('Int64')

# Impute missing arrival delays with carrier+month+hour average
flights_imp['arr_delay'] = flights_imp['arr_delay'].fillna(
    flights_imp.groupby(['carrier', 'month', 'dep_hour'])['arr_delay'].transform('mean')
)

# Optional: fill any still-missing entries with overall mean
flights_imp['arr_delay'] = flights_imp['arr_delay'].fillna(flights_imp['arr_delay'].mean())

print(f"There are {flights_imp['arr_delay'].isna().sum()} missing values left in the dataset.")


There are 0 missing values left in the dataset.


### Part B(iii) (1 mark)

Using the `describe` method from Pandas, discuss if the distribution (quantiles, min, max) of the values in `arr_delay` has changed after imputation.

## Part C

You are asked to study the size of planes appearing in `flights`. 

### Part C(i) (1 mark)

Add a new column named `size_category` to `planes` using the `apply` function from Pandas. The new column should indicate if a plane is `Small` (less than 100 seats), `Medium` (between 100 and 180 seats), 'Large' (between 181 and 250 seats) or 'Jumbo' (more than 250 seats). Keep the `%%timeit` at the beginning of the code cell below to measure the speed of executing your code.

In [2]:
%%timeit 


UsageError: %%timeit is a cell magic, but the cell body is empty. Did you mean the line magic %timeit (single %)?


### Part C(ii) (2 mark)

Repeat the task from C(i) but using only Numpy operations, to add a new column `size_category_np` to `planes`. Again, use `%%timeit` when running your code. What do you notice in comparison to the result in C(i) in terms of the running time?

In [ ]:
%%timeit


### Part C(iii) (1 mark)

Using the `merge` method from Pandas, add the `size_category` column from `planes` to `flights`. 

### Part C(iv) (1 marks)

Determine for each size category the destination (`dest`) with the most flights. Which carrier uses the most `Large` planes?